# Writing a Public DID to the Sovrin StagingNet

### This notebook walks through how to connect an Aca-Py agent to a live Indy network. In our example we will be connecting to the Sovrin Stagingnet. To view details about Sovrin Stagingnet transactions please visit [Indyscan](https://indyscan.io/home/SOVRIN_STAGINGNET).

## 1. Initialise a controller for Issuer Agent

In [ ]:
%autoawait
import time
import asyncio
import sys
import pprint
from aiohttp import ClientResponseError

from aries_basic_controller.aries_controller import AriesAgentController
# Create a small utility to print json formatted outout more human-readable    
pp = pprint.PrettyPrinter(indent=4)
    
WEBHOOK_HOST = "0.0.0.0"
WEBHOOK_BASE = ""

WEBHOOK_PORT = 8052
ADMIN_URL = "http://external-agent:8051"

In [ ]:
# Based on the aca-py agent you wish to control
agent_controller = AriesAgentController(webhook_host=WEBHOOK_HOST, webhook_port=WEBHOOK_PORT,
                                       webhook_base=WEBHOOK_BASE, admin_url=ADMIN_URL)

## 2. Get current public DID

Before being able to write to any indy based ledger, your agent must have a public DID written on the ledger giving it the authority to write to it. As the cell below shows, this agent does not currently have a public DID. So any writes to the ledger will be rejected.

In [ ]:
try:
    response = await agent_controller.wallet.get_public_did()
    print(response)
except ConnectionrefusedError as err:
    print(err)

### THIS WILL FAIL

Try it and look at the error message that is printed. It will give you useful information as to why it fails, in case it is unclear to you at this point why this should fail.

In [ ]:
schema_id = 'MGgoJXWbeupKsaHDa7s4fW:2:testabc:0.0.1'
try:
    response = await agent_controller.definitions.write_cred_def(schema_id)
except ClientResponseError as err:
    print(err)
    

## 3. Generate a new DID

Before being able to write a DID to the ledger, you must create one using the wallet api. This api returns the identifier (the DID), and the verification key for that DID. A representation of it's public key. 

In [ ]:
# generate new DID
response = await agent_controller.wallet.create_did()

try:
    did_object = response['result']
    print("New DID", did_object)
except:
    print("Unexpected error:", sys.exc_info()[0])
    # If you wish to get the error stack trace uncomment the line below and run it again
    # raise

## 4. Write DID to Sovrin Stagingnet

Anoyone can write a DID to the Sovrin StagingNet, it is a permissionless ledger. 

Visit [Sovrin Selfserve Portal](https://selfserve.sovrin.org) for more information. We have provided an automated process to write DIDs to Stagingnet in the step below.

In [ ]:
# write new DID to Sovrin Stagingnet
import requests
import json 

url = 'https://selfserve.sovrin.org/nym'

payload = {"network":"stagingnet","did": did_object["did"],"verkey":did_object["verkey"],"paymentaddr":""}

# Adding empty header as parameters are being sent in payload
headers = {}

try:
    r = requests.post(url, data=json.dumps(payload), headers=headers)
    print(r.json())
    print(r.status_code)
except:
    print("Unexpected error:", sys.exc_info()[0])


## 5. Accepting the Transaction Author Agreement (TAA)

Although the Sovrin StagingNet is permissionless, before DIDs have the authority to write to the ledger they must accept something called a transaction author agreement by signing it using the DID they have on the ledger.

As a global public ledger, the Sovrin Ledger and all its participants are subject to privacy and data protection regulations such as the EU General Data Protection Regulation (GDPR). These regulations require that the participants be explicit about responsibilities for Personal Data.

To clarify these responsibilities and provide protection for all parties, the Sovrin Governance Framework Working Group developed an agreement between Transaction Authors and the Sovrin Foundation. The TAA can be found at Sovrin.org. It ensures that users are aware of and consent to the fact that all data written to the Sovrin Ledger cannot be removed, even if the original author of the transaction requests its removal.

The TAA outlines the policies that users must follow when interacting with the Sovrin Ledger. When a user’s client software is preparing a transaction for submission to the network, it must include a demonstration that the user had the opportunity to review the current TAA and accept it. This is done by including some additional fields in the ledger write transaction: 

* A hash of the agreement
* A date when the agreement was accepted, and
* A string indicating the user interaction that was followed to obtain the acceptance.

The Indy client API used by Sovrin has been extended to allow users to review current and past agreements and to indicate acceptance through an approved user interaction pattern. - source: https://sovrin.org/preparing-for-the-sovrin-transaction-author-agreement/

For more details on TAA please read more at the following links:
* [Preparing for the Sovrin Transaction Author Agreement](https://sovrin.org/preparing-for-the-sovrin-transaction-author-agreement/)
* [How the recent approval of the Sovrin Governance Framework v2 affects Transaction Authors
](https://sovrin.org/how-the-recent-approval-of-the-sovrin-governance-framework-v2-affects-transaction-authors/)
* [TAA v2](https://github.com/sovrin-foundation/sovrin/blob/master/TAA/TAA.md)
* [TAA Acceptance Mechanism List (AML)](https://github.com/sovrin-foundation/sovrin/blob/master/TAA/AML.md)

In [ ]:
# This will not work until you have accepted the TAA -Error expected
# Look at the error message the code below produces. It will tell you the same
try:
    response = await agent_controller.wallet.assign_public_did(did_object["did"])
except ClientResponseError as err:
    print(err)

### Let's obtain a TAA

In [ ]:
try:
    response = await agent_controller.ledger.get_taa()
    TAA = response['result']['taa_record']
    TAA['mechanism'] = "service_agreement"
    print(TAA)
except:
    print("Unexpected error:", sys.exc_info()[0])
    raise

### Now we can accept it
We assume you want this 

In [ ]:
try:
    response = await agent_controller.ledger.accept_taa(TAA)
    ## Will return {} if successful
    print(response)
except:
    print("Unexpected error:", sys.exc_info()[0])
    raise

## 6. Set public DID

Now you are able to assign the DID written to the ledger as public.

In [ ]:
try:
    response = await agent_controller.wallet.assign_public_did(did_object["did"])
    pp.pprint(response)
except:
    print("Unexpected error:", sys.exc_info()[0])
#     raise

## 7. Get public DID

In [ ]:
try:
    response = await agent_controller.wallet.get_public_did()
    pp.pprint(response)
    print("\n")
    issuer_nym = response['result']['did']
    pp.pprint("Issuer public DID:  " + issuer_nym)
except:
    print("Unexpected error:", sys.exc_info()[0])
    raise

## 8. Fetch verkey for public DID

Additionally, we can verify that this DID does actually resolve to the public key material on the ledger.

In [ ]:
try:
    issuer_verkey = await agent_controller.ledger.get_did_verkey(issuer_nym)
    print(issuer_verkey)
except:
    print("Unexpected error:", sys.exc_info()[0])
    raise

## 9. Get public DID endpoint

As well as providing a publically accessible endpoint to contact the DID controller through the agent framework.

In [ ]:
try:
    issuer_endpoint = await agent_controller.ledger.get_did_endpoint(issuer_nym)
    print(issuer_endpoint)
except:
    print("Unexpected error:", sys.exc_info()[0])
    raise

## 10. End of Tutorial

Be sure to terminate the controller so you can run another tutorial.

In [ ]:
await agent_controller.terminate()

# Proceed to Part 3 on [Issuer Notebook](http://localhost:8889/lab/tree/Part%203%20-%20Issue%20Credential.ipynb)

Here you will be issued with a credential into your mobile SSI wallet that will be verified in part 4.